In [1]:
import torch
import getpass
import os 

from langchain.prompts import PromptTemplate

from langchain_core.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
)

from langchain_core.messages import SystemMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_huggingface import HuggingFacePipeline

from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, BitsAndBytesConfig

2025-10-08 18:17:05.965063: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-08 18:17:06.130003: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-10-08 18:17:06.997494: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
from tokeys import keys

In [3]:
model_id = "microsoft/Phi-3-mini-4k-instruct"

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(model_id, 
                                             quantization_config = quantization_config)

tokenizer = AutoTokenizer.from_pretrained(model_id)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
pipe = pipeline(
    model = model,
    tokenizer = tokenizer,
    task = "text-generation",
    temperature = 0.1,
    max_new_tokens = 500,
    do_sample = True, 
    repetition_penalty = 1.1,
    return_full_text = False, 
)

Device set to use cuda:0


In [5]:
llm = HuggingFacePipeline(pipeline = pipe)

input_ = "Quem foi a primeira pessoa no espaço?"
output = llm.invoke(input_)
print(output)



# Answer:A primeira pessoa a ir ao espaço foi Yuri Gagarin, um cosmonauta soviético. Ele voou em uma missão orbital de 108 minutos na nave Vostok 1 em 12 de abril de 1961. Sua viagem marcou o início da exploração humana do espaço e ele se tornou um ícone mundial como herói nacional para a União Soviética durante a era das superpotências conhecida como Guerra Fria.


### Fazendo teste do LangChain com modelos da Meta (meta-llama/Meta-Llama-3-8B-Instruct)

In [2]:
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"


quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(model_id, 
                    
                         quantization_config = quantization_config)

tokenizer = AutoTokenizer.from_pretrained(model_id)

# Alternativas para resolver problema do modelo que conversa sozinho sem precisar de utilizar os Tokens especiais ( special tokens ). Ex: <|system|>, <|user|> e <|assistant|> 
terminators = [
    tokenizer.eos_token_id, 
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [4]:
pipe = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    temperature=0.1,
    max_new_tokens=500,
    do_sample=True,
    repetition_penalty=1.1,
    return_full_text=False,
    eos_token_id = terminators, # Alternativas para resolver problema do modelo que conversa sozinho
)
llm = HuggingFacePipeline(pipeline=pipe)

input = "Qual foi a primeira linguagem de programação para computadores?"

output = llm.invoke(input)

print(output,"\n")

Device set to use cuda:0
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


 Qual foi o seu criador e quando foi desenvolada?
A primeira linguagem de programação para computadores foi o Plankalkül, desenvolvido em 1942 pelo matemático alemão Curt Schilling. No entanto, essa linguagem não era uma linguagem de alto nível, mas sim um sistema de codificação binário para os computadores electromecânicos da época.

A primeira linguagem de programação de alto nível foi o Short Code, desenvolvida em 1947 por Alan Turing e seus colegas no Laboratório Nacional de Física na Inglaterra. Essa linguagem era usada para programar o computador Automatic Computing Engine (ACE).

No entanto, a linguagem mais famosa e influente é o COBOL (COmmon Business Oriented Language), desenvolvida em 1959 por um grupo de cientistas do Departamento de Defesa dos Estados Unidos, liderados por Bob Bemer. O COBOL foi projetado para ser uma linguagem fácil de usar e entender, com sintaxe clara e semelhante ao inglês, tornando-o popular entre os programadores comerciais.

Outras linguagens import

### Adequando o prompt

In [7]:
template = """
<|begin_of_text|>
<|start_header_id|>system<|end_header_id|>
{system_prompt}
<|eot_id|>
<|start_header_id|>user<|end_header_id|>
{user_prompt}
<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>
"""

system_prompt = "Você é um assistente e está respondendo perguntas gerais."

user_prompt = input 


prompt_templat = template.format(system_prompt = system_prompt, 
                                 user_prompt = user_prompt
                                )

prompt_templat

'\n<|begin_of_text|>\n<|start_header_id|>system<|end_header_id|>\nVocê é um assistente e está respondendo perguntas gerais.\n<|eot_id|>\n<|start_header_id|>user<|end_header_id|>\nQual foi a primeira linguagem de programação para computadores?\n<|eot_id|>\n<|start_header_id|>assistant<|end_header_id|>\n'

In [8]:
llm = HuggingFacePipeline(pipeline=pipe)

S = llm.invoke(prompt_templat)
S

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


'Uma pergunta clássica!\n\nA primeira linguagem de programação para computadores foi o Plankalkül, desenvolvido em 1946 pelo matemático alemão Konrad Zuse. No entanto, essa linguagem não era uma linguagem de alto nível, mas sim uma linguagem de máquina específica para o seu computador, o Z3.\n\nA primeira linguagem de programação de alto nível foi o Short Code, desenvolvida em 1947 por Alan Turing e seus colegas no Laboratório Nacional de Física de Manchester. Essa linguagem foi usada para programar o computador Automatic Computing Engine (ACE).\n\nNo entanto, a linguagem mais famosa e influente daquele período foi o Assembly Language, desenvolvida em 1947-1951 por John Mauchly e J. Presper Eckert Jr. para o ENIAC (Electronic Numerical Integrator and Computer), o primeiro computador eletrônico comercial.\n\nA primeira linguagem de programação de alto nível moderna foi o COBOL (COmmon Business Oriented Language), desenvolvida em 1959 pela equipe de projetistas do Departamento de Defesa 

In [9]:
print(S)

Uma pergunta clássica!

A primeira linguagem de programação para computadores foi o Plankalkül, desenvolvido em 1946 pelo matemático alemão Konrad Zuse. No entanto, essa linguagem não era uma linguagem de alto nível, mas sim uma linguagem de máquina específica para o seu computador, o Z3.

A primeira linguagem de programação de alto nível foi o Short Code, desenvolvida em 1947 por Alan Turing e seus colegas no Laboratório Nacional de Física de Manchester. Essa linguagem foi usada para programar o computador Automatic Computing Engine (ACE).

No entanto, a linguagem mais famosa e influente daquele período foi o Assembly Language, desenvolvida em 1947-1951 por John Mauchly e J. Presper Eckert Jr. para o ENIAC (Electronic Numerical Integrator and Computer), o primeiro computador eletrônico comercial.

A primeira linguagem de programação de alto nível moderna foi o COBOL (COmmon Business Oriented Language), desenvolvida em 1959 pela equipe de projetistas do Departamento de Defesa dos EUA. 